In [1]:
import sqlite3
import random

In [2]:
conn = sqlite3.connect('example.db')

In [3]:
cursor = conn.cursor()

In [4]:
pages_table_with_index = """
create table if not exists pages (
page_id int not null primary key,
page_name varchar(20))
"""


pages_table_without_index = """
create table if not exists pages_without_index (
page_id int not null,
page_name varchar(20))
"""

page_likes = """
create table if not exists page_likes (
user_id int not null,
page_id int not null
)"""

index_query = """ CREATE INDEX page_likes_page_id ON page_likes (page_id); """


page_likes_without_index = """
create table if not exists page_likes_without_index (
user_id int not null,
page_id int not null
)"""

In [5]:
cursor.execute(pages_table_with_index)
cursor.execute(pages_table_without_index)
cursor.execute(page_likes)
cursor.execute(index_query)
cursor.execute(page_likes_without_index)

In [6]:
conn.commit()

In [7]:
for i in range(1, 10):
    insert_pages = f"insert into pages values ({i}, 'page_name')"
    cursor.execute(insert_pages)
conn.commit()

for i in range(10):
    insert_page_likes = f"insert into page_likes values ({random.randint(1,100)},{random.randint(1,10)})"
    cursor.execute(insert_page_likes)
conn.commit()

In [8]:
cursor.execute("select * from pages;")

In [9]:
for i in cursor.fetchall():
    print(i)

(1, 'page_name')
(2, 'page_name')
(3, 'page_name')
(4, 'page_name')
(5, 'page_name')
(6, 'page_name')
(7, 'page_name')
(8, 'page_name')
(9, 'page_name')


In [10]:
cursor.execute("select distinct page_id from page_likes")
for i in cursor.fetchall():
    print(i)

(1,)
(2,)
(3,)
(4,)
(5,)
(6,)
(8,)
(10,)


# Solution 1 

In [11]:
solution = """SELECT page_id
FROM pages
EXCEPT
SELECT page_id
FROM page_likes; """


cursor.execute(solution)
for i in cursor.fetchall():
    print(i)

(7,)
(9,)


# Solution 2

In [12]:
solution = """SELECT page_id
FROM pages
where page_id not in (SELECT page_id
FROM page_likes)
; """


cursor.execute(solution)
for i in cursor.fetchall():
    print(i)

(7,)
(9,)


# Solution 3

In [13]:
solution = """SELECT pages.page_id
FROM pages left join page_likes
on pages.page_id = page_likes.page_id
where page_likes.page_id is null
; """


cursor.execute(solution)
for i in cursor.fetchall():
    print(i)

(7,)
(9,)


# Solution 4

In [14]:
solution = """SELECT pages.page_id
FROM pages 
where not exists (select 1 from page_likes
                           where page_likes.page_id = pages.page_id)
; """


cursor.execute(solution)
for i in cursor.fetchall():
    print(i)

(7,)
(9,)


# Solution 4 Efficient

In [15]:
solution = """SELECT pages.page_id
FROM pages 
where not exists (select 1 from page_likes
                           where page_likes.page_id = pages.page_id
                           limit 1)
; """


cursor.execute(solution)
for i in cursor.fetchall():
    print(i)

(7,)
(9,)


In [16]:
conn.close()